<a href="https://colab.research.google.com/github/Yagami360/MachineLearning_Exercises_Python_PyTorch/blob/master/GAN_WGAN_PyTorch/WGAN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Colab 用の Jupiter Notebook

In [0]:
# -*- coding:utf-8 -*-

"""
    更新情報
    [19/04/23] : 新規作成
    [xx/xx/xx] : 
               : 
"""
import os
import numpy as np
from tqdm import tqdm

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image


class Generator( nn.Module ):
    """
    WGAN の生成器 G [Generator] 側のネットワーク構成を記述したモデル。

    [public]
    [protected] 変数名の前にアンダースコア _ を付ける
        _device : <toech.cuda.device> 使用デバイス
        _layer : <nn.Sequential> 生成器のネットワーク構成
    [private] 変数名の前にダブルアンダースコア __ を付ける（Pythonルール）

    """
    def __init__(
        self,
        device,
        n_input_noize_z = 100,
        n_channels = 3,
        n_fmaps = 64
    ):
        super( Generator, self ).__init__()
        self._device = device
        
        self._layer = nn.Sequential(
            nn.ConvTranspose2d(n_input_noize_z, n_fmaps*8, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(n_fmaps*8),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d( n_fmaps*8, n_fmaps*4, kernel_size=4, stride=2, padding=1, bias=False ),
            nn.BatchNorm2d(n_fmaps*4),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d( n_fmaps*4, n_fmaps*2, kernel_size=4, stride=2, padding=1, bias=False ),
            nn.BatchNorm2d(n_fmaps*2),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d( n_fmaps*2, n_fmaps, kernel_size=4, stride=2, padding=1, bias=False ),
            nn.BatchNorm2d(n_fmaps),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d( n_fmaps, n_channels, kernel_size=4, stride=2, padding=1, bias=False ),
            nn.Tanh()
        ).to( self._device )

        self.init_weight()
        return

    def init_weight( self ):
        """
        独自の重みの初期化処理
        """
        return

    def forward( self, input ):
        """
        ネットワークの順方向での更新処理
        ・nn.Module クラスのメソッドをオーバライト

        [Args]
            input : <Tensor> ネットワークに入力されるデータ（ノイズデータ等）
        [Returns]
            output : <Tensor> ネットワークからのテンソルの出力
        """
        output = self._layer(input)
        return output


class Critic( nn.Module ):
    """
    WGAN のクリティック側のネットワーク構成を記述したモデル。

    [public]
    [protected] 変数名の前にアンダースコア _ を付ける
        _device : <toech.cuda.device> 使用デバイス
        _layer : <nn.Sequential> クリティックのネットワーク構成
    [private] 変数名の前にダブルアンダースコア __ を付ける（Pythonルール）
    """
    def __init__(
       self,
       device,
       n_channels = 3,
       n_fmaps = 64
    ):
        super( Critic, self ).__init__()
        self._device = device
        
        self._layer = nn.Sequential(
            nn.Conv2d(n_channels, n_fmaps, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(n_fmaps, n_fmaps*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_fmaps*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(n_fmaps*2, n_fmaps*4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_fmaps*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(n_fmaps*4, n_fmaps*8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_fmaps*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(n_fmaps*8, 1, kernel_size=4, stride=1, padding=0, bias=False),
        ).to( self._device )

        self.init_weight()

        return

    def init_weight( self ):
        """
        独自の重みの初期化処理
        """
        return

    def forward(self, input):
        # input : torch.Size([batch_size, n_channels, width, height])
        output = self._layer( input )

        # ミニバッチ平均
        output = output.mean(0)

        return output.view(1)


class WassersteinGAN( object ):
    """
    WGAN [Wasserstein GAN] を表すクラス
    --------------------------------------------
    [public]

    [protected] 変数名の前にアンダースコア _ を付ける
        _device : <toech.cuda.device> 実行デバイス

        _n_epoches : <int> エポック数（学習回数）
        _learnig_rate : <float> 最適化アルゴリズムの学習率
        _batch_size : <int> ミニバッチ学習時のバッチサイズ
        _n_channels : <int> 入力画像のチャンネル数
        _n_fmaps : <int> 特徴マップの枚数
        _n_input_noize_z : <int> 入力ノイズ z の次元数

        _n_critic : <int> クリティックの更新回数
        _w_clamp_lower : <float> 重みクリッピングの下限値
        _w_clamp_upper : <float> 重みクリッピングの上限値

        _generator : <nn.Module> DCGAN の生成器
        _critic : <nn.Module> DCGAN のクリティック

        _loss_fn : <> 損失関数

        _G_optimizer : <torch.optim.Optimizer> 生成器の最適化アルゴリズム
        _C_optimizer : <torch.optim.Optimizer> クリティックの最適化アルゴリズム

        _loss_G_historys : <list> 生成器 G の損失関数値の履歴（イテレーション毎）
        _loss_C_historys : <list> クリティック C の損失関数値の履歴（イテレーション毎）

        _images_historys : <list> 生成画像のリスト

        _f_historys : <list> クリティックからの出力（リプシッツ連続な関数 f）

    [private] 変数名の前にダブルアンダースコア __ を付ける（Pythonルール）

    """
    def __init__(
        self,
        device,
        n_epoches = 300,
        learing_rate = 0.0001,
        batch_size = 64,
        n_channels = 3,
        n_fmaps = 64,
        n_input_noize_z = 100,
        n_critic = 5,
        w_clamp_lower = - 0.01,
        w_clamp_upper = 0.01
    ):
        self._device = device

        self._n_epoches = n_epoches
        self._learning_rate = learing_rate
        self._batch_size = batch_size
        self._n_channels = n_channels
        self._n_fmaps = n_fmaps
        self._n_input_noize_z = n_input_noize_z

        self._n_critic = n_critic
        self._w_clamp_lower = w_clamp_lower
        self._w_clamp_upper = w_clamp_upper

        self._generator = None
        self._critic = None
        self._loss_fn = None
        self._G_optimizer = None
        self._C_optimizer = None
        self._loss_G_historys = []
        self._loss_C_historys = []
        self._images_historys = []

        self.model()
        self.loss()
        self.optimizer()
        return

    def print( self, str = "" ):
        print( "----------------------------------" )
        print( "WassersteinGAN" )
        print( self )
        print( str )
        print( "_device :", self._device )
        print( "_n_epoches :", self._n_epoches )
        print( "_learning_rate :", self._learning_rate )
        print( "_batch_size :", self._batch_size )
        print( "_n_channels :", self._n_channels )
        print( "_n_fmaps :", self._n_fmaps )
        print( "_n_input_noize_z :", self._n_input_noize_z )
        print( "_n_critic :", self._n_critic )
        print( "_w_clamp_lower :", self._w_clamp_lower )
        print( "_w_clamp_upper :", self._w_clamp_upper )
        print( "_generator :", self._generator )
        print( "_critic :", self._critic )
        print( "_loss_fn :", self._loss_fn )
        print( "_G_optimizer :", self._G_optimizer )
        print( "_C_optimizer :", self._C_optimizer )
        print( "----------------------------------" )
        return


    @property
    def device( self ):
        """ device の Getter """
        return self._device

    @device.setter
    def device( self, device ):
        """ device の Setter """
        self._device = device
        return

    @property
    def loss_G_history( self ):
        return self._loss_G_historys

    @property
    def loss_C_history( self ):
        return self._loss_C_historys

    @property
    def images_historys( self ):
        return self._images_historys


    def model( self ):
        """
        モデルの定義を行う。

        [Args]
        [Returns]
        """
        self._generator = Generator( 
            self._device, 
            n_input_noize_z = self._n_input_noize_z,
            n_channels = self._n_channels,
            n_fmaps = self._n_fmaps
        )

        self._critic = Critic( 
            self._device,
            n_channels = self._n_channels,
            n_fmaps = self._n_fmaps
        )

        return

    def loss( self ):
        """
        損失関数の設定を行う。
        [Args]
        [Returns]
        """
        # Binary Cross Entropy
        # L(x,y) = - { y*log(x) + (1-y)*log(1-x) }
        # x,y の設定は、後の fit() 内で行う。
        #self._loss_fn = nn.BCELoss()

        return

    def optimizer( self ):
        """
        モデルの最適化アルゴリズムの設定を行う。
        [Args]
        [Returns]
        """
        # GeneratorとDiscriminatorはそれぞれ別のOptimizerがある
        # PyTorchはOptimizerの更新対象となるパラメータを第1引数で指定することになっている（TensorFlowやKerasにはなかった）
        # この機能のおかげで D_optimizer.step() でパラメータ更新を走らせたときに、
        # Discriminatorのパラメータしか更新されず、Generatorのパラメータは固定される。
        # これにより TensorFlow における、
        # tf.control_dependencies(...) : sess.run で実行する際のトレーニングステップの依存関係（順序）を定義
        # に対応する処理が簡単にわかりやすく行える。
        """
        self._G_optimizer = optim.Adam(
            params = self._generator.parameters(),
            lr = self._learning_rate,
            betas = (0.5,0.999)
        )
        """
        self._G_optimizer = optim.RMSprop(
            params = self._generator.parameters(),
            lr = self._learning_rate
        )

        """
        self._C_optimizer = optim.Adam(
            params = self._critic.parameters(),
            lr = self._learning_rate,
            betas = (0.5,0.999)
        )
        """
        self._C_optimizer = optim.RMSprop(
            params = self._critic.parameters(),
            lr = self._learning_rate
        )
        return


    def fit( self, dloader, n_sava_step = 5 ):
        """
        指定されたトレーニングデータで、モデルの fitting 処理を行う。
        [Args]
            dloader : <DataLoader> 学習用データセットの DataLoader
            n_sava_step : <int> 学習途中での生成画像の保存間隔（エポック単位）
        [Returns]
        """
        # 1 と -1 の Tensor : 誤差逆伝搬の基準に使用する。
        one_tsr =  torch.FloatTensor([1]).to( self._device )
        mone_tsr = one_tsr * -1

        # 入力ノイズ z
        input_noize_z = torch.FloatTensor( self._batch_size, self._n_input_noize_z ).to( self._device )

        #-------------------------------------
        # モデルを学習モードに切り替える。
        #-------------------------------------
        self._generator.train()
        self._critic.train()

        #-------------------------------------
        # 学習処理ループ
        #-------------------------------------
        iterations = 0      # 学習処理のイテレーション回数

        print("Starting Training Loop...")
        # エポック数分トレーニング
        for epoch in tqdm( range(self._n_epoches), desc = "Epoches" ):
            # DataLoader から 1minibatch 分取り出し、ミニバッチ処理
            for (images,targets) in tqdm( dloader, desc = "minbatch process in DataLoader" ):
                #print( "images.size() : ", images.size() )
                #print( "targets.size() : ", targets.size() )

                # 一番最後のミニバッチループで、バッチサイズに満たない場合は無視する
                # （後の計算で、shape の不一致をおこすため）
                if images.size()[0] != self._batch_size:
                    break

                iterations += 1

                # ミニバッチデータを GPU へ転送
                images = images.to( self._device )

                #====================================================
                # クリティック C の fitting 処理
                #====================================================
                # 無効化していたクリティック C のネットワークの勾配計算を有効化。
                for param in self._critic.parameters():
                    param.requires_grad = True

                for n in range( self._n_critic ):
                    #----------------------------------------------------
                    # 重みクリッピング
                    #----------------------------------------------------
                    for param in self._critic.parameters():
                        #print( "critic param :", param )
                        param.data.clamp_( self._w_clamp_lower, self._w_clamp_upper )
                        #print( "critic param :", param )

                    # 生成器 G に入力するノイズ z
                    # Generatorの更新の前にノイズを新しく生成しなおす必要があり。
                    input_noize_z.resize_( self._batch_size, self._n_input_noize_z, 1 , 1 ).normal_(0, 1)

                    #----------------------------------------------------
                    # 勾配を 0 に初期化
                    # （この初期化処理が必要なのは、勾配がイテレーション毎に加算される仕様のため）
                    #----------------------------------------------------
                    self._C_optimizer.zero_grad()

                    #----------------------------------------------------
                    # 学習用データをモデルに流し込む
                    # model(引数) で呼び出せるのは、__call__ をオーバライトしているため
                    #----------------------------------------------------
                    # E[C(x)] : 本物画像 x = image を入力したときのクリティックの出力 （平均化処理済み）
                    C_x = self._critic( images )
                    #print( "C_x.size() :", C_x.size() )
                    #print( "C_x :", C_x )

                    # 微分を行わない処理の範囲を with 構文で囲む
                    # クリティック D の学習中は、生成器 G のネットワークの勾配は更新しない。
                    with torch.no_grad():
                        # G(z) : 生成器から出力される偽物画像
                        G_z = self._generator( input_noize_z )
                        #print( "G_z.size() :", G_z.size() )     # torch.Size([128, 1, 28, 28])
                        #print( "G_z :", G_z )

                    # E[ C( G(z) ) ] : 偽物画像を入力したときの識別器の出力 (平均化処理済み)
                    C_G_z = self._critic( G_z )
                    #print( "C_G_z.size() :", C_G_z.size() )
                    #print( "C_G_z :", C_G_z )

                    #----------------------------------------------------
                    # 損失関数を計算する
                    # 出力と教師データを損失関数に設定し、誤差 loss を計算
                    # この設定は、損失関数を __call__ をオーバライト
                    # loss は Pytorch の Variable として帰ってくるので、これをloss.data[0]で数値として見る必要があり
                    #----------------------------------------------------
                    # E_x[ C(x) ]
                    #loss_C_real = torch.mean( C_x )
                    loss_C_real = C_x
                    #print( "loss_C_real : ", loss_C_real.item() )

                    # E_z[ C(G(z) ]
                    #loss_C_fake = torch.mean( C_G_z )
                    loss_C_fake = C_G_z
                    #print( "loss_C_fake : ", loss_C_fake.item() )

                    # クリティック C の損失関数 = E_x[ C(x) ] + E_z[ C(G(z) ]
                    loss_C = loss_C_real - loss_C_fake
                    #print( "loss_C : ", loss_C.item() )

                    #----------------------------------------------------
                    # 誤差逆伝搬
                    #----------------------------------------------------
                    loss_C_real.backward( one_tsr )
                    loss_C_fake.backward( mone_tsr )
                    #loss_C.backward()

                    #----------------------------------------------------
                    # backward() で計算した勾配を元に、設定した optimizer に従って、重みを更新
                    #----------------------------------------------------
                    self._C_optimizer.step()

                #====================================================
                # 生成器 G の fitting 処理
                #====================================================
                # クリティック C のネットワークの勾配計算を行わないようにする。
                for param in self._critic.parameters():
                    param.requires_grad = False

                # 生成器 G に入力するノイズ z
                # Generatorの更新の前にノイズを新しく生成しなおす必要があり。
                input_noize_z.resize_( self._batch_size, self._n_input_noize_z, 1, 1 ).normal_(0, 1)

                #----------------------------------------------------
                # 勾配を 0 に初期化
                # （この初期化処理が必要なのは、勾配がイテレーション毎に加算される仕様のため）
                #----------------------------------------------------
                self._G_optimizer.zero_grad()

                #----------------------------------------------------
                # 学習用データをモデルに流し込む
                # model(引数) で呼び出せるのは、__call__ をオーバライトしているため
                #----------------------------------------------------
                # G(z) : 生成器から出力される偽物画像
                G_z = self._generator( input_noize_z )
                #print( "G_z.size() :", G_z.size() )
                #print( "G_z :", G_z )

                # E[C( G(z) )] : 偽物画像を入力したときのクリティックの出力（平均化処理済み）
                C_G_z = self._critic( G_z )
                #print( "C_G_z.size() :", C_G_z.size() )

                #----------------------------------------------------
                # 損失関数を計算する
                #----------------------------------------------------
                # L_G = E_z[ C(G(z) ]
                #loss_G = torch.mean( C_G_z )
                loss_G = C_G_z
                #print( "loss_G :", loss_G )

                #----------------------------------------------------
                # 誤差逆伝搬
                #----------------------------------------------------
                #loss_G.backward()
                loss_G.backward( one_tsr )

                #----------------------------------------------------
                # backward() で計算した勾配を元に、設定した optimizer に従って、重みを更新
                #----------------------------------------------------
                self._G_optimizer.step()

                #
                self._loss_C_historys.append( loss_C.item() )
                self._loss_G_historys.append( loss_G.item() )

            #----------------------------------------------------
            # 学習過程での自動生成画像
            #----------------------------------------------------
            # 特定のエポックでGeneratorから画像を保存
            if( epoch % n_sava_step == 0 ):
                images = self.generate_images( n_samples = 64, b_transformed = False )
                self._images_historys.append( images )

                save_image( 
                    tensor = images, 
                    filename = "WGAN_Image_epoches{}_iters{}.png".format( epoch, iterations )
                )

        print("Finished Training Loop.")
        return


    def generate_images( self, n_samples = 64, b_transformed = False ):
        """
        DCGAN の Generator から、画像データを自動生成する。
        [Input]
            n_samples : <int> 生成する画像の枚数
            b_transformed : <bool> 画像のフォーマットを Tensor から変換するか否か
        [Output]
            images : <Tensor> / shape = [n_samples, n_channels, height, width]
                生成された画像データのリスト
                行成分は生成する画像の数 n_samples
        """
        # 生成器を推論モードに切り替える。
        self._generator.eval()

        # 生成のもとになる乱数を生成
        input_noize_z = torch.rand( (n_samples, self._n_input_noize_z, 1, 1) ).to( self._device )

        # 画像を生成
        images = self._generator( input_noize_z )
        #print( "images.size() :", images.size() )   # torch.Size([64, 1, 28, 28])

        if( b_transformed == True ):
            # Tensor → numpy に変換
            images = images.cpu().detach().numpy()

        return images


In [0]:
# -*- coding:utf-8 -*-

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
#import pickle
import scipy.misc

# PyTorch
import torch
from torch.utils.data import TensorDataset, DataLoader

import torchvision      # 画像処理関連
import torchvision.transforms as transforms
from torchvision.utils import save_image


#--------------------------------
# 設定可能な定数
#--------------------------------
#DEVICE = "CPU"               # 使用デバイス ("CPU" or "GPU")
DEVICE = "GPU"                # 使用デバイス ("CPU" or "GPU")
DATASET = "MNIST"            # データセットの種類（"MNIST" or "CIFAR-10"）
#DATASET = "CIFAR-10"          # データセットの種類（"MNIST" or "CIFAR-10"）
DATASET_PATH = "./dataset"    # 学習用データセットへのパス
NUM_SAVE_STEP = 1             # 自動生成画像の保存間隔（エポック単位）

NUM_EPOCHES = 10               # エポック数（学習回数）
LEARNING_RATE = 0.00005       # 学習率 (Default:0.00005)
BATCH_SIZE = 64               # ミニバッチサイズ
IMAGE_SIZE = 64               # 入力画像のサイズ（pixel単位）
NUM_CHANNELS = 1              # 入力画像のチャンネル数
NUM_FEATURE_MAPS = 64         # 特徴マップの枚数
NUM_INPUT_NOIZE_Z = 100       # 生成器に入力するノイズ z の次数
NUM_CRITIC = 5                # クリティックの更新回数
WEIGHT_CLAMP_LOWER = - 0.01   # 重みクリッピングの下限値
WEIGHT_CLAMP_UPPER = 0.01     # 重みクリッピングの上限値


def main():
    """
    Wasserstein による画像の自動生成
    ・学習用データセットは、MNIST / CIFAR-10
    """
    print("Start main()")
    
    # バージョン確認
    print( "PyTorch :", torch.__version__ )

    # 実行条件の出力
    print( "----------------------------------------------" )
    print( "実行条件" )
    print( "----------------------------------------------" )
    print( "開始時間：", datetime.now() )
    print( "DEVICE : ", DEVICE )
    print( "NUM_EPOCHES : ", NUM_EPOCHES )
    print( "LEARNING_RATE : ", LEARNING_RATE )
    print( "BATCH_SIZE : ", BATCH_SIZE )
    print( "IMAGE_SIZE : ", IMAGE_SIZE )
    print( "NUM_CHANNELS : ", NUM_CHANNELS )
    print( "NUM_FEATURE_MAPS : ", NUM_FEATURE_MAPS )
    print( "NUM_INPUT_NOIZE_Z : ", NUM_INPUT_NOIZE_Z )
    print( "NUM_CRITIC : ", NUM_CRITIC )
    print( "WEIGHT_CLAMP_LOWER : ", WEIGHT_CLAMP_LOWER )
    print( "WEIGHT_CLAMP_UPPER : ", WEIGHT_CLAMP_UPPER )

    #===================================
    # 実行 Device の設定
    #===================================
    if( DEVICE == "GPU" ):
        use_cuda = torch.cuda.is_available()
        if( use_cuda == True ):
            device = torch.device( "cuda" )
            print( "実行デバイス :", device)
            print( "GPU名 :", torch.cuda.get_device_name(0))
            print("torch.cuda.current_device() =", torch.cuda.current_device())
        else:
            print( "can't using gpu." )
            device = torch.device( "cpu" )
            print( "実行デバイス :", device)
    else:
        device = torch.device( "cpu" )
        print( "実行デバイス :", device)

    print( "----------------------------------------------" )

    # seed 値の固定
    import random
    random.seed(8)
    np.random.seed(8)
    torch.manual_seed(8)

    #======================================================================
    # データセットを読み込み or 生成
    # データの前処理
    #======================================================================
    dataset = DATASET

    # データをロードした後に行う各種前処理の関数を構成を指定する。
    if( dataset == "MNIST" ):
        """
        transform = transforms.Compose(
            [
                transforms.ToTensor(),   # Tensor に変換]
            ]
        """
        transform = transforms.Compose(
            [
                transforms.Scale(IMAGE_SIZE),
                transforms.ToTensor(),   # Tensor に変換]
                transforms.Normalize((0.5,), (0.5,)),   # 1 channel 分
            ]
        )
    elif( dataset == "CIFAR-10" ):
        transform = transforms.Compose(
            [
                transforms.Scale(IMAGE_SIZE),
                transforms.ToTensor(),   # Tensor に変換
                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
            ]
        )
    else:
        print( "Warning: Invalid dataset" )

    #---------------------------------------------------------------
    # data と label をセットにした TensorDataSet の作成
    #---------------------------------------------------------------
    if( dataset == "MNIST" ):
        ds_train = torchvision.datasets.MNIST(
            root = DATASET_PATH,
            train = True,
            transform = transform,      # transforms.Compose(...) で作った前処理の一連の流れ
            target_transform = None,    
            download = True,
        )

        ds_test = torchvision.datasets.MNIST(
            root = DATASET_PATH,
            train = False,
            transform = transform,
            target_transform = None,
            download = True
        )
    elif( dataset == "CIFAR-10" ):
        ds_train = torchvision.datasets.CIFAR10(
            root = DATASET_PATH,
            train = True,
            transform = transform,      # transforms.Compose(...) で作った前処理の一連の流れ
            target_transform = None,    
            download = True
        )

        ds_test = torchvision.datasets.CIFAR10(
            root = DATASET_PATH,
            train = False,
            transform = transform,
            target_transform = None,
            download = True
        )
    else:
        print( "WARNING: Inavlid dataset" )

    print( "ds_train :", ds_train ) # MNIST : torch.Size([60000, 28, 28]) , CIFAR-10 : (50000, 32, 32, 3)
    print( "ds_test :", ds_test )

    #---------------------------------------------------------------
    # TensorDataset → DataLoader への変換
    # DataLoader に変換することで、バッチ処理が行える。
    # DataLoader クラスは、dataset と sampler クラスを持つ。
    # sampler クラスには、ランダムサンプリングや重点サンプリングなどがある
    #---------------------------------------------------------------
    dloader_train = DataLoader(
        dataset = ds_train,
        batch_size = BATCH_SIZE,
        shuffle = True
    )

    dloader_test = DataLoader(
        dataset = ds_test,
        batch_size = BATCH_SIZE,
        shuffle = False
    )
    
    # [MNIST]
    # Number of datapoints: 60000
    # dloader_train.datset
    # dloader_train.sampler = <RandomSampler, len() = 60000>
    print( "dloader_train :", dloader_train )
    print( "dloader_test :", dloader_test )
    
    #======================================================================
    # モデルの構造を定義する。
    #======================================================================
    if( dataset == "MNIST" ):
        """
        model = WassersteinGANforMNIST(
            device = device,
            n_epoches = NUM_EPOCHES,
            learing_rate = LEARNING_RATE,
            batch_size = BATCH_SIZE,
            n_input_noize_z = NUM_INPUT_NOIZE_Z,
            n_critic = NUM_CRITIC,
            w_clamp_lower = WEIGHT_CLAMP_LOWER,
            w_clamp_upper = WEIGHT_CLAMP_UPPER
        )
        """
        model = WassersteinGAN(
            device = device,
            n_epoches = NUM_EPOCHES,
            learing_rate = LEARNING_RATE,
            batch_size = BATCH_SIZE,
            n_channels = NUM_CHANNELS,
            n_fmaps = NUM_FEATURE_MAPS,
            n_input_noize_z = NUM_INPUT_NOIZE_Z,
            n_critic = NUM_CRITIC,
            w_clamp_lower = WEIGHT_CLAMP_LOWER,
            w_clamp_upper = WEIGHT_CLAMP_UPPER
        )

    else:
        model = WassersteinGAN(
            device = device,
            n_epoches = NUM_EPOCHES,
            learing_rate = LEARNING_RATE,
            batch_size = BATCH_SIZE,
            n_channels = NUM_CHANNELS,
            n_fmaps = NUM_FEATURE_MAPS,
            n_input_noize_z = NUM_INPUT_NOIZE_Z,
            n_critic = NUM_CRITIC,
            w_clamp_lower = WEIGHT_CLAMP_LOWER,
            w_clamp_upper = WEIGHT_CLAMP_UPPER
        )

    model.print( "after init()" )

    #print( "model.device() :", model.device )

    #---------------------------------------------------------------
    # 損失関数を設定
    #---------------------------------------------------------------
    #model.loss()

    #---------------------------------------------------------------
    # optimizer を設定
    #---------------------------------------------------------------
    #model.optimizer()

    #======================================================================
    # モデルの学習フェイズ
    #======================================================================
    model.fit( dloader = dloader_train, n_sava_step = NUM_SAVE_STEP )

    #===================================
    # 学習結果の描写処理
    #===================================
    #-----------------------------------
    # 損失関数の plot
    #-----------------------------------
    plt.clf()
    plt.plot(
        range( 0, len(model.loss_G_history) ), model.loss_G_history,
        label = "loss : Generator",
        linestyle = '-',
        linewidth = 0.2,
        color = 'red'
    )
    plt.plot(
        range( 0, len(model.loss_C_history) ), model.loss_C_history,
        label = "loss : Critic",
        linestyle = '-',
        linewidth = 0.2,
        color = 'blue'
    )
    plt.title( "loss" )
    plt.legend( loc = 'best' )
    #plt.xlim( 0, len(model.loss_G_history) )
    #plt.ylim( [0, 1.05] )
    plt.xlabel( "iterations" )
    plt.grid()
    plt.tight_layout()
    plt.savefig(
        "WGAN_Loss_epoches{}_lr{}_batchsize{}.png".format( NUM_EPOCHES, LEARNING_RATE, BATCH_SIZE ),  
        dpi = 300, bbox_inches = "tight"
    )
    plt.show()

    #-------------------------------------------------------------------
    # 学習済み DCGAN に対し、自動生成画像を表示
    #-------------------------------------------------------------------
    images = model.generate_images( n_samples = 64, b_transformed = False )
    #print( "images.size() : ", images.size() )    # (64, 1, 28, 28)

    save_image( 
        tensor = images, 
        filename = "WGAN_Image_epoches{}_lr{}_batchsize{}.png".format( NUM_EPOCHES, LEARNING_RATE, BATCH_SIZE )
    )

    """
    images = model.generate_images( n_samples = 64, b_transformed = True )
    scipy.misc.imsave( 
        "DCGAN_Image_epoches{}_lr{}_batchsize{}.png".format( NUM_EPOCHES, LEARNING_RATE, BATCH_SIZE ),
        np.vstack(
            np.array( [ np.hstack(img) for img in images ] )
        )
    )
    """

    print("Finish main()")
    print( "終了時間：", datetime.now() )

    return


    
if __name__ == '__main__':
     main()

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:208: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
0it [00:00, ?it/s]

Start main()
PyTorch : 1.0.1.post2
----------------------------------------------
実行条件
----------------------------------------------
開始時間： 2019-04-24 12:59:12.475658
DEVICE :  GPU
NUM_EPOCHES :  10
LEARNING_RATE :  5e-05
BATCH_SIZE :  64
IMAGE_SIZE :  64
NUM_CHANNELS :  1
NUM_FEATURE_MAPS :  64
NUM_INPUT_NOIZE_Z :  100
NUM_CRITIC :  5
WEIGHT_CLAMP_LOWER :  -0.01
WEIGHT_CLAMP_UPPER :  0.01
実行デバイス : cuda
GPU名 : Tesla T4
torch.cuda.current_device() = 0
----------------------------------------------


9920512it [00:00, 20189476.62it/s]                            


Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 311744.15it/s]                           
0it [00:00, ?it/s]

...

1654784it [00:00, 5648471.89it/s]                           
8192it [00:00, 126928.28it/s]


Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
ds_train : Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./dataset
    Transforms (if any): Compose(
                             Scale(size=64, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=(0.5,), std=(0.5,))
                         )
    Target Transforms (if any): None
ds_test : Dataset MNIST
    Number of datapoints: 10000
    Split: test
    Root Location: ./dataset
    Transforms (if any): Compose(
                             Scale(size=64, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=(0.5,), std=(0.5,))
                         )
    Target Transforms (if any): None
dloader_train : <torch.utils.data.dataloader.DataLoader object at 0x7f6054eb3e48>
dloader_test : <torc

minbatch process in DataLoader:   0%|          | 0/938 [00:00<?, ?it/s]

----------------------------------
WassersteinGAN
after init()
_device : cuda
_n_epoches : 10
_learning_rate : 5e-05
_batch_size : 64
_n_channels : 1
_n_fmaps : 64
_n_input_noize_z : 100
_n_critic : 5
_w_clamp_lower : -0.01
_w_clamp_upper : 0.01
_generator : Generator(
  (_layer): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias


Epoches:  10%|█         | 1/10 [05:45<51:48, 345.40s/it]

minbatch process in DataLoader:   0%|          | 0/938 [00:00<?, ?it/s]

minbatch process in DataLoader:   0%|          | 1/938 [00:00<05:27,  2.86it/s]

minbatch process in DataLoader:   0%|          | 2/938 [00:00<05:32,  2.82it/s]

minbatch process in DataLoader:   0%|          | 3/938 [00:01<05:35,  2.78it/s]

minbatch process in DataLoader:   0%|          | 4/938 [00:01<05:37,  2.77it/s]

minbatch process in DataLoader:   1%|          | 5/938 [00:01<05:38,  2.75it/s]

minbatch process in DataLoader:   1%|          | 6/938 [00:02<05:39,  2.75it/s]

minbatch process in DataLoader:   1%|          | 7/938 [00:02<05:39,  2.74it/s]

minbatch process in DataLoader:   1%|          | 8/938 [00:02<05:40,  2.73it/s]

minbatch process in DataLoader:   1%|          | 9/938 [00:03<05:40,  2.73it/s]

minbatch process in DataLoader:   1%|          | 10/938 [00:03<05:40,  2.73it/s]

minbatch process in DataLoader:   1%|          | 11/938 [